_Adapted from notebook used by [nbclab.github.io](https://nbclab.github.io)._

# Retrieve new publications from PubMed 

This notebook is used to search for and retrieve latest publications by Dr. Khan using BioPython's PubMed search tool. A publication-specific MarkDown file is generated for each unique paper, with many elements automatically set up. As noted in the original notebook, you generally should check that the link to the markdown file exists. Unfortunately, preprints cannot be found via this method (though they can be added manually). This notebook cannot find new preprints. The process is automated and runs monthly using Github actions.

## Steps (via Github or manual)

1. Run this notebook.
2. If any new papers were grabbed, check the following:
    1. The paper has either of the lab PIs as an author. Ensure that it isn't by *another* AR Khan.
    2. The paper is not a duplicate of a preprint or another version of the paper. If so, merge the two versions.
3. Save the changes to the notebook.
4. Push changes to the notebook and affected files to GitHub.
5. Open a pull request to khanlab/khanlab.github.io

In [1]:
# Libraries
import os 

from Bio import Entrez, Medline
from datetime import datetime
from dateutil import parser
import pandas as pd

### Check existing publications

In [2]:
# First count number of articles from previous grab
pub_data = "_data/publications/publications.csv"

# Update count of publications from existing file
old_count = 0 
if os.path.isfile(pub_data):
    df_old = pd.read_csv(pub_data)
    old_count = len(df_old)

### Perform new query

In [3]:
# Only grab papers from after the lab PI came to UWO
search_criteria = ['''"Khan AR"[AUTH] AND ("2015/01/01"[PDAT] : "3000/12/31"[PDAT]) AND
                    ("Western University"[AFFL] OR "University of Western Ontario"[AFFL] OR
                     "Brain and Mind Institute"[AFFL] OR "Robarts Research Institute"[AFFL])''']

# Email required to search
Entrez.email = ''

In [4]:
rows = []

# Publications to skip (possibly due to another user with same initial)
skip_pmids = [32971934, 29641820, 29634829]
skip_pmids = [str(pmid) for pmid in skip_pmids]

for TERM in search_criteria:
    search = Entrez.esearch(db="pubmed", retmax="2", term=TERM)
    result = Entrez.read(search)
    print(f"Total number of publications containing {TERM}: {result['Count']}")
    
    search_all = Entrez.esearch(db="pubmed", term=TERM, retmax=result["Count"])
    result_all = Entrez.read(search_all)
    ids_all = result_all['IdList']
    pubs_all = Entrez.efetch(db="pubmed", id=ids_all, rettype='medline', retmode='text')
    records = Medline.parse(pubs_all)
    
    acceptable_formats = ["journal article", "comparative study", "editorial"]
    
    for record in records:
        if any([type_.lower() in acceptable_formats for type_ in record.get('PT')]):
            pmid = record.get("PMID")
            pmcid = record.get("PMC", "")
            
            doi = [aid for aid in record.get("AID", []) if aid.endswith(" [doi]")]
            if doi:
                doi = doi[0].replace(" [doi]", "")
            else:
                doi = ""
            
            title = record.get("TI").rstrip(".")
            authors = record.get("AU")
            
            pub_date = parser.parse(record.get("DP"))
            journal = record.get('TA')
            volume = record.get('VI', '')
            issue = record.get('IP', '')
            pages = record.get('PG', '')
            
            row = [pmid, pmcid, doi, title, authors, pub_date.year, pub_date.month,
                   pub_date.day, journal, volume, issue, pages]
            rows += [row]
            
df = pd.DataFrame(columns=['pmid', 'pmcid', 'doi', 'title', 'authors',
                           'year', 'month', 'day',
                           'journal', 'volume', 'issue', 'pages'],
                  data=rows)
df = df[~df["pmid"].isin(skip_pmids)]

new_pubs = df[~df['pmid'].isin(df_old['pmid'])]

# Append to old pubs to solve date issue
df = df_old.append(new_pubs)
df = df.sort_values(by=['year', 'month', 'day'], ascending=False)
df = df.fillna('')

# Save all relevant info from articles to a csv.
print("Saving identified publications to csv...")
df.to_csv('_data/publications/publications.csv', index=False)

Total number of publications containing "Khan AR"[AUTH] AND ("2015/01/01"[PDAT] : "3000/12/31"[PDAT]) AND
                    ("Western University"[AFFL] OR "University of Western Ontario"[AFFL] OR
                     "Brain and Mind Institute"[AFFL] OR "Robarts Research Institute"[AFFL]): 63


Saving identified publications to csv...


In [5]:
# Add papers we already have pages for.
if len(skip_pmids) > 0:
    for pmid in skip_pmids:
        df = df[df['pmid'] != pmid]
        
print(f"{len(df)} total articles found.")
print(f"{len(df) - old_count} new articles found.")

print("\nNew publications found:")
for _, pub in new_pubs.iterrows():
    print(f"Title: {pub['title']} ({pub['journal']})")
    print(f"Authors: {pub['authors']}")
    print(f"Journal (Date): {pub['journal']} ({pub['month']}/{pub['day']}/{pub['year']})\n")


115 total articles found.
58 new articles found.

New publications found:
Title: Automated hippocampal unfolding for morphometry and subfield segmentation with HippUnfold (Elife)
Authors: ['DeKraker J', 'Haast RAM', 'Yousif MD', 'Karat B', 'Lau JC', 'Kohler S', 'Khan AR']
Journal (Date): Elife (12/15/2022)

Title: The role of the temporal pole in temporal lobe epilepsy: A diffusion kurtosis imaging study (Neuroimage Clin)
Authors: ['Kasa LW', 'Peters T', 'Mirsattari SM', 'Jurkiewicz MT', 'Khan AR', 'A M Haast R']
Journal (Date): Neuroimage Clin (2/1/2022)

Title: Microstructural imaging and transcriptomics of the basal forebrain in first-episode psychosis (Transl Psychiatry)
Authors: ['Park MTM', 'Jeon P', 'French L', 'Dempster K', 'Chakravarty MM', 'MacKinley M', 'Richard J', 'Khan AR', 'Theberge J', 'Palaniyappan L']
Journal (Date): Transl Psychiatry (9/1/2022)

Title: qMRI-BIDS: An extension to the brain imaging data structure for quantitative magnetic resonance imaging data (Sci Da

In [6]:
# Also output the df in case output limit exceeded
new_pubs

,pmid,pmcid,doi,title,authors,year,month,day,journal,volume,issue,pages
0,36519725,PMC9831605,10.7554/eLife.77945,Automated hippocampal unfolding for morphometr...,"[DeKraker J, Haast RAM, Yousif MD, Karat B, La...",2022,12,15,Elife,11,,
1,36126518,PMC9486670,10.1016/j.nicl.2022.103201,The role of the temporal pole in temporal lobe...,"[Kasa LW, Peters T, Mirsattari SM, Jurkiewicz ...",2022,2,1,Neuroimage Clin,36,,103201
2,36050318,PMC9436926,10.1038/s41398-022-02136-0,Microstructural imaging and transcriptomics of...,"[Park MTM, Jeon P, French L, Dempster K, Chakr...",2022,9,1,Transl Psychiatry,12,1,358
3,36002444,PMC9402561,10.1038/s41597-022-01571-4,qMRI-BIDS: An extension to the brain imaging d...,"[Karakuzu A, Appelhoff S, Auer T, Boudreau M, ...",2022,8,24,Sci Data,9,1,517
4,35961469,,10.1016/j.neuroimage.2022.119553,Mapping the subcortical connectome using in vi...,"[Kai J, Khan AR, Haast RA, Lau JC]",2022,11,15,Neuroimage,262,,119553
5,35776355,,10.1007/978-1-0716-2409-8_14,Three-Dimensional Atlas of the Human Amygdala ...,"[Seguin D, Khan AR, Duerden EG]",2022,2,1,Methods Mol Biol,2515,,227-236
6,35766286,PMC9242645,10.7554/eLife.75769,Waveform detection by deep learning reveals mu...,"[Mofrad MH, Gilmore G, Koller D, Mirsattari SM...",2022,6,29,Elife,11,,
7,35516811,PMC9063519,10.3389/fnins.2022.871228,Microscopy-BIDS: An Extension to the Brain Ima...,"[Bourget MH, Kamentsky L, Ghosh SS, Mazzamuto ...",2022,2,1,Front Neurosci,16,,871228
8,35464316,PMC9023071,10.3389/fnins.2022.833209,Enabling Complex Fibre Geometries Using 3D Pri...,"[Kuehn TK, Mushtaha FN, Khan AR, Baron CA]",2022,2,1,Front Neurosci,16,,833209
9,35311645,PMC9020819,10.7554/eLife.66884,Activity in perirhinal and entorhinal cortex p...,"[Ferko KM, Blumenthal A, Martin CB, Proklova D...",2022,3,21,Elife,11,,
